# Preprocessing Notebook for Passive Acoustic Monitoring (PAM)

This notebook guides through the preprocessing steps for the PAM project, including loading audio files, generating mel spectrograms, and saving processed data.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display

# Parameters for audio processing
sample_rate = 22050  # Audio sampling rate (Hz)
clip_duration = 5    # Duration of each clip in seconds
samples_per_clip = sample_rate * clip_duration

# Path to audio data and metadata
audio_dir = '../data/raw_audio/'         # directory containing audio files
metadata_file = '../data/train_metadata.csv'  # CSV with columns ['filename', 'species']

# Load metadata
metadata = pd.read_csv(metadata_file)  # expects columns 'filename' and 'species'

# Initialize lists to hold data and labels
X = []   # will hold mel spectrogram arrays
y = []   # will hold corresponding species labels

# Process each audio file
for idx, row in metadata.iterrows():
    file_path = os.path.join(audio_dir, row['filename'])
    if not os.path.exists(file_path):
        continue  # skip missing files
    # Load the full audio file
    signal, sr = librosa.load(file_path, sr=sample_rate)
    # Split into non-overlapping clips of fixed duration
    for start in range(0, len(signal), samples_per_clip):
        clip = signal[start:start + samples_per_clip]
        # If clip is too short, skip it
        if len(clip) < samples_per_clip:
            continue
        # Compute the mel spectrogram (128 mel bands)
        mel_spec = librosa.feature.melspectrogram(
            y=clip, sr=sr, n_mels=128, hop_length=512
        )
        # Convert to log scale (decibels) for better representation
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        # Append to our dataset
        X.append(mel_spec_db)
        y.append(row['species'])

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Save mel spectrograms and labels
output_dir = '../data/processed/'
os.makedirs(os.path.join(output_dir, 'mel_spectrograms'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

# Save mel spectrograms as images
for i in range(len(X)):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(X[i], sr=sample_rate, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Mel Spectrogram - {y[i]}')
    plt.savefig(os.path.join(output_dir, 'mel_spectrograms', f'{i}_{y[i]}.png'))
    plt.close()

# Save labels to a text file
np.savetxt(os.path.join(output_dir, 'labels', 'labels.txt'), y, fmt='%s')

# Display completion message
print('Preprocessing complete! Mel spectrograms and labels have been saved.')